# Testing for painter image prediction

In [ ]:
import os
import numpy as np
import pandas as pd

from helpers.preprocessing_helpers import *

In [ ]:
from PIL import Image
import glob

origin_dir = "data/raw"
new_dir = "data/preprocessed"
#painters = ["Mondriaan", "Picasso", "Rubens"]
painters = ["Picasso", "Rubens"]

im = Image.open(f"{origin_dir}/{'Picasso'}/0-picassofull-1.JPG")
# im.show()
im.close()

In [ ]:
# image_list = []
# valid_images = [".jpg",".png",".jpeg"]
# painter = painters[1]
# orig_path = f"{origin_dir}/{painter}/"
# new_path = f"{new_dir}/{painter}/"

# for f in os.listdir(orig_path):
#     ext = os.path.splitext(f)[1]
#     if ext.lower() not in valid_images:
#         print(f)
#         continue
#     #print(f"saving {f} to {new_path}")
#     img = Image.open(os.path.join(orig_path,f))
#     cover = img.resize((180, 180))
#     #cover.save(os.path.join(new_path,f))
# # print(type(image_list))    

In [ ]:
try:
    list = get_raw_file_name_for_painter("no")
    list
except FileNotFoundError:
    print("correct error caught")

In [ ]:
list = get_raw_file_name_for_painter(painters[0])


In [ ]:

# make_data_sets(painters, 600) 
# write an fuction that loop over an list

In [ ]:
list = [1, 2, 3, 6, 5, 6, 7, 8, 9]
print(list[6:9], list[-3:])

In [ ]:
800//6*4+800//6*1+800//6*1  

In [ ]:
demo = "test.png"
pre, _ = os.path.splitext(demo)
#os.rename(renamee, pre + new_extension)
pre

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

new_base_dir = "data/preprocessed/"
train_dataset = image_dataset_from_directory(
    os.path.join(new_base_dir, "train"),
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    os.path.join(new_base_dir, "val"),
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    os.path.join(new_base_dir, "test"),
    image_size=(180, 180),
    batch_size=32)

In [ ]:
print(type(train_dataset.class_names))
print(type(validation_dataset))
print(type(test_dataset))

In [ ]:
train_dataset.class_names

# Scraping

## Functions

In [ ]:
from bs4 import BeautifulSoup
import requests
from PIL import Image
import os
import uuid

PATH_TO_SAVE = "data/raw/Rembrandt"


In [ ]:
def check_dir(dir:str):
    if not os.path.isdir(dir):
        os.makedirs(dir)

def download_image(pic_url:str, saveName:str):
    with open(saveName, 'wb') as handle:
        response = requests.get(pic_url, stream=True)
        if not response.ok:
            pass
        for block in response.iter_content(1024):
            if not block:
                break
            handle.write(block)

def download_image_from_site_default(uri:str):
    r = requests.get(uri)
    s = BeautifulSoup(r.text, "html.parser")
    part = s.find(id="workimage")
    src = part.find("img")['src']
    pic_url = uri[:uri.rfind("/")] + "/" + src
    saveName = PATH_TO_SAVE +pic_url[pic_url.rfind("/"):]
    # if os.path.exists(saveName):
    #     name, ext = os.path.splitext(saveName)
    #     name += str(uuid.uuid4())
    #     saveName = name + ext
    download_image(pic_url, saveName)

def download_image_from_site_try_2(uri:str):
    r = requests.get(uri)
    s = BeautifulSoup(r.text, "html.parser")
    images = s.find_all("img")
    images = str(images)
    
    start = images.find('src="images/')
    images = images[start+5:]
    end = images.find('"')
    src = images[:end]
    
    pic_url = uri[:uri.rfind("/")] + "/" + src
    saveName = PATH_TO_SAVE +pic_url[pic_url.rfind("/"):]
    # if os.path.exists(saveName):
    #     name, ext = os.path.splitext(saveName)
    #     name += str(uuid.uuid4())
    #     saveName = name + ext
    download_image(pic_url, saveName)

In [ ]:
def get_menu_items(uri:str):
    links = []
    r = requests.get(uri)
    s = BeautifulSoup(r.text, "html.parser")
    #  Looking for the table with the classes 'wikitable' and 'sortable'
    table = s.find('table', class_='tablelinks')
    i =0
    if table == None:
        return None

    for row in table.find_all('tr'): 
        columns = row.find_all('td')
        td = columns[0]
        td = str(td)
        start = td.find('href="')
        td = td[start+6:]
        end = td.find('"')
        img_page_link = td[:end]
        if img_page_link != 0: 
            i+=1
            links.append(img_page_link)
    return links

def get_table(uri:str):# Creating list with all tables
    links = []
    r = requests.get(uri)
    s = BeautifulSoup(r.text, "html.parser")

    #  Looking for the table with the classes 'wikitable' and 'sortable'
    table = s.find('table', class_='tablelinks')
    i =0
    if table == None:
        return None

    for row in table.find_all('tr'): 
        columns = row.find_all('td')
        td = columns[1]
        td = str(td)
        start = td.find('href="')
        td = td[start+6:]
        end = td.find('"')
        img_page_link = td[:end]
        if img_page_link != 0: 
            i+=1
            links.append(img_page_link)
        # print(img_page_link)

    return links


## Get images

In [ ]:
base_url = "http://www.rembrandtpainting.net/"
start_uri = "http://www.rembrandtpainting.net/complete_catalogue/complete_catalogue.htm"

In [ ]:
menu_items = get_menu_items(start_uri)
menu_items

In [ ]:
for page in menu_items:
    new_base = "http://www.rembrandtpainting.net/complete_catalogue/"
    uri = new_base+page

    table = get_table(uri)
    check_dir(PATH_TO_SAVE)
    done = []
    for i in table:
        if new_base+i in done:
            pass
        else:
            done.append(new_base+i)
            try:
                download_image_from_site_default(new_base+i)
            except:
                try: 
                    download_image_from_site_try_2(new_base+i)
                except:
                    print(f"error at: {new_base+i}")


In [ ]:
for filename in os.listdir(PATH_TO_SAVE):
    try:
        img = Image.open(PATH_TO_SAVE+"/"+filename) # open the image file
        img.verify() # verify that it is, in fact an image
        img.close()
    except (IOError, SyntaxError) as e:
        print('Bad file:', filename) # print out the names of corrupt files
        os.remove(PATH_TO_SAVE+"/"+filename)

In [ ]:
print(f"aantal images: {len(os.listdir(PATH_TO_SAVE))}")

# test

In [ ]:
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Activation
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [ ]:
batch_size = 64
epochs=15
image_shape = (180, 180)
PREPROCESSING_FOLDER = "data/preprocessed"
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                f"{PREPROCESSING_FOLDER}/train", image_size=image_shape, batch_size=batch_size, label_mode='categorical'
                )
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                f"{PREPROCESSING_FOLDER}/test", image_size=image_shape, batch_size=batch_size, label_mode='categorical'
                )
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                f"{PREPROCESSING_FOLDER}/val", image_size=image_shape, batch_size=batch_size, label_mode='categorical'
                )

In [ ]:
train_dataset

In [ ]:
output_shape = len(train_dataset.class_names)
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)

x = layers.Dropout(0.5)(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(output_shape, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

filename = "test.keras"
callback = [keras.callbacks.ModelCheckpoint(filename, save_best_only=True)]

model.compile(
    loss="categorical_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)


In [ ]:
model.summary()

In [ ]:
print(train_dataset, type(train_dataset))
print(val_dataset, type(val_dataset))
print(batch_size, type(batch_size))
print(epochs, type(epochs))
print(callback, type(callback))


In [ ]:
history=model.fit(
        train_dataset, 
        batch_size=batch_size, epochs=epochs, 
        validation_data=(val_dataset), 
        callbacks=callback
        )

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 180  # figure size vergroten

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(epochs, accuracy, "bo", label="Training accuracy")
ax1.plot(epochs, val_accuracy, "b", label="Validation accuracy")
ax1.set_title("Accuracy")
ax1.legend();

ax2.plot(epochs, loss, "bo", label="Training loss")
ax2.plot(epochs, val_loss, "b", label="Validation loss")
ax2.set_title("Loss")
ax2.legend();

In [ ]:
model.evaluate(test_dataset)

# Model testing

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Activation
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [ ]:
batch_size = 64
image_shape = (180, 180)
PREPROCESSING_FOLDER = "data/preprocessed"
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                f"{PREPROCESSING_FOLDER}/test", image_size=image_shape, batch_size=batch_size, label_mode='categorical'
                )

In [ ]:
input_shape = (180, 180, 3)
output_shape = 3

data_augmentation = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.2),
            ])
        
conv_base = keras.applications.vgg19.VGG19(
            weights="imagenet",
            include_top=False
            )

conv_base.trainable = True
for layer in conv_base.layers[:-2]:
    layer.trainable = False


inputs = keras.Input(shape=input_shape)
x = inputs
x = data_augmentation(x) 

x = keras.applications.vgg19.preprocess_input(x)
x = conv_base(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(output_shape, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
model.load_weights("painter_baseline.keras")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
filename = "zeeuws_meisje_piet_mondriaan.jpg"
filename, _ = os.path.splitext(f'static/preprocessed/{filename}')

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                f"{PREPROCESSING_FOLDER}/test", image_size=image_shape, batch_size=batch_size, label_mode='categorical'
                )
from PIL import Image
import numpy as np

img = Image.open(filename + ".png")

numpydata = np.asarray(img)
numpydata = np.expand_dims(numpydata, axis = 0)

In [ ]:
img = tf.keras.utils.load_img(
    filename + ".png", target_size=(180, 180)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [ ]:
#predictions = model.predict(numpydata)
predictions = model.predict(img_array)

In [ ]:
test_dataset.class_names

In [ ]:
score = tf.nn.softmax(predictions[0])

class_names = test_dataset.class_names

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

